In [25]:
import numpy as np
from scipy.fft import dctn, idctn

In [39]:
def mdct4(x):
    N = x.shape[0]
    if N%4 != 0:
        raise ValueError("MDCT4 only defined for vectors of length multiple of four.")
    M = N // 2
    N4 = N // 4
    
    rot = np.roll(x, N4)
    rot[:N4] = -rot[:N4]
    t = np.arange(0, N4)
    w = np.exp(-1j*2*np.pi*(t + 1./8.) / N)
    c = np.take(rot,2*t) - np.take(rot, N-2*t-1) \
        - 1j * (np.take(rot, M+2*t) - np.take(rot,M-2*t-1))
    c = (2./np.sqrt(N)) * w * np.fft.fft(0.5 * c * w, N4)
    y = np.zeros(M)
    y[2*t] = np.real(c[t])
    y[M-2*t-1] = -np.imag(c[t])
    return y


def imdct4(x):
    N = x.shape[0]
    if N%2 != 0:
        raise ValueError("iMDCT4 only defined for even-length vectors.")
    M = N // 2
    N2 = N*2
    
    t = np.arange(0,M)
    w = np.exp(-1j*2*np.pi*(t + 1./8.) / N2)
    c = np.take(x,2*t) + 1j * np.take(x,N-2*t-1)
    c = 0.5 * w * c
    c = np.fft.fft(c,M)
    c = ((8 / np.sqrt(N2))*w)*c
    
    rot = np.zeros(N2)
    
    rot[2*t] = np.real(c[t])
    rot[N+2*t] = np.imag(c[t])
    
    t = np.arange(1,N2,2)
    rot[t] = -rot[N2-t-1]
    
    t = np.arange(0,3*M)
    y = np.zeros(N2)
    y[t] = rot[t+M]
    t = np.arange(3*M,N2)
    y[t] = -rot[t-3*M]
    return y

In [49]:
def dctf(x):
    N = x.shape[0]

    if N%4 != 0:
        raise ValueError("MDCT4 only defined for vectors of length multiple of four.")

    M = N // 2
    N4 = N // 4

    a = x[0*N4:1*N4]
    b = x[1*N4:2*N4]
    c = x[2*N4:3*N4]
    d = x[3*N4:4*N4]

    br = np.flip(b)
    cr = np.flip(c)

    dct(np.hstack([-cr - d, a - br]))


    p = -np.flip(c) - d
    q = a - np.flip(b)

    pq = np.hstack([p, q])

    return dctn(pq, norm='ortho', axes=0)

z = np.random.randn(64)

%timeit dctf(z)
%timeit mdct4(z)

18.7 µs ± 1.45 µs per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
54 µs ± 357 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [52]:
def mill(a, b, c, d):
    return f'({a} - {b}r)(-{a}r + {b})({c} + {d}r)({c}r + {d})'

print(mill('a', 'b', 'c', 'd'))
print(mill('c', 'd', 'e', 'f'))
print(mill('e', 'f', 'g', 'h'))



(a - br)(-ar + b)(c + dr)(cr + d)
(c - dr)(-cr + d)(e + fr)(er + f)
(e - fr)(-er + f)(g + hr)(gr + h)


In [53]:
import tensorflow as tf


tf.signal.kaiser_bessel_derived_window(16)


2022-12-07 15:42:11.064148: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-07 15:42:12.400826: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-07 15:42:12.401710: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


<tf.Tensor: shape=(16,), dtype=float32, numpy=
array([0.00429333, 0.08631644, 0.287716  , 0.57040375, 0.82136446,
       0.9577158 , 0.9962678 , 0.99999076, 0.99999076, 0.9962678 ,
       0.9577158 , 0.82136446, 0.57040375, 0.287716  , 0.08631644,
       0.00429333], dtype=float32)>

In [54]:
from scipy import signal

In [58]:
kaiser = signal.windows.kaiser_bessel_derived(2*N, n16)

print(N)

kaiser.shape

NameError: name 'n16' is not defined